In [1]:
import pandas as pd
from tqdm import tqdm

## Load Clusters and Alarms

In [2]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [2]:
mob = pd.read_parquet("alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [3]:
adsl = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")

In [4]:
ptn = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [5]:
sdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [6]:
pdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [9]:
mob = mob[["alarm_id", "last_occurrence"]]

In [11]:
adsl = adsl[["alarm_id", "last_occurrence"]]
ptn = ptn[["alarm_id", "last_occurrence"]]
sdh = sdh[["alarm_id", "last_occurrence"]]
pdh = pdh[["alarm_id", "last_occurrence"]]


In [12]:
alarms = pd.concat([mob, adsl, ptn, sdh, pdh])


In [13]:
alarms.to_parquet("alarms datasets/alarms.parquet")

## Clusters Statistics

In [30]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")
alarms = pd.read_parquet("alarms datasets/alarms.parquet")

In [31]:
clusters_filtered = clusters[["alarm_id", "cluster_id", "cluster_id2","first_occurrence"]]

In [35]:
clusters_complete = clusters_filtered.merge(alarms, on="alarm_id", how="left")

In [44]:
clusters_complete["first_occurrence"] = pd.to_datetime(clusters_complete["first_occurrence"])

In [54]:
clusters_complete = clusters_complete.dropna(subset=["last_occurrence"])

In [56]:
clusters_complete.to_parquet("real-time clusters/clusters_complete.parquet")

### Clusters Size

Number of alarms within a cluster

In [6]:
clusters_complete = pd.read_parquet("real-time clusters/clusters_complete.parquet")

In [7]:
clusters_grouped_by_id = clusters_complete.groupby(["cluster_id", "cluster_id2"])

In [12]:
clusters_grouped = clusters_grouped_by_id 

In [13]:
size_df = clusters_grouped_by_id.size().reset_index(name='size')

In [14]:
size_df.to_parquet("real-time clusters/size_df.parquet")

### Clusters Lifespan

Time difference from the last last_occurrence and the first first_occurrence of alarms 

In [73]:

def calculate_lifespan(group):
    return group['last_occurrence'].max() - group['first_occurrence'].min()

tqdm.pandas()


lifespan_df = clusters_grouped.apply(calculate_lifespan, include_groups=False).reset_index(name='lifespan')


In [75]:
lifespan_df.to_parquet("real-time clusters/lifespan_df.parquet")

### Clusters Delta First Occurrence

Distance between the last available first_occurrence and the first

In [76]:


def calculate_lifespan(group):
    return group['first_occurrence'].max() - group['first_occurrence'].min()


tqdm.pandas()


delta_df = clusters_grouped.progress_apply(calculate_lifespan).reset_index(name='delta first occurrence')

100%|██████████| 6369371/6369371 [17:32<00:00, 6048.92it/s]


In [77]:
delta_df

,cluster_id,cluster_id2,lifespan
0,202301010000_1,202301010000_33,0 days
1,202301010000_10,202301010000_133,0 days
2,202301010000_104,202301010000_79,0 days
3,202301010000_105,202301010000_80,0 days
4,202301010000_106,202301010000_81,0 days
...,...,...,...
6369366,202312312358_5,202312312354_5,0 days
6369367,202312312358_6,202312312354_5,0 days
6369368,202312312358_7,202312312354_5,0 days
6369369,202312312358_8,202312312354_7,0 days


In [78]:
delta_df.to_parquet("real-time clusters/delta_df.parquet")

In [3]:
delta_df = pd.read_parquet("real-time clusters/delta_df.parquet")

In [5]:
delta_df.sort_values(by="delta first occurrence", ascending=False)

,cluster_id,cluster_id2,lifespan
5187223,202311041120_19,202311041120_6,0 days 02:07:54
4355824,202309272328_4,202309272328_3,0 days 01:53:46
3079985,202307270332_16,202307270332_10,0 days 01:34:24
5710744,202311272202_10,202311272202_3,0 days 01:32:50
399034,202302020154_50,202302020154_8,0 days 01:22:56
...,...,...,...
2419111,202306300446_37,202306300434_9,0 days 00:00:00
2419110,202306300446_36,202306300434_9,0 days 00:00:00
2419109,202306300446_35,202306300434_9,0 days 00:00:00
2419105,202306300446_31,202306300442_9,0 days 00:00:00


## Merge Statistics

In [15]:
size_df = pd.read_parquet("real-time clusters/size_df.parquet")
lifespan_df = pd.read_parquet("real-time clusters/lifespan_df.parquet")
delta_df = pd.read_parquet("real-time clusters/delta_df.parquet")

In [18]:
clusters_statistics = size_df.merge(lifespan_df, on=["cluster_id", "cluster_id2"], how="left").merge(delta_df, on=["cluster_id", "cluster_id2"], how="left")

In [19]:
clusters_statistics

,cluster_id,cluster_id2,size,lifespan_x,lifespan_y
0,202301010000_1,202301010000_33,1,1 days 23:25:34,0 days
1,202301010000_10,202301010000_133,1,0 days 01:14:29,0 days
2,202301010000_104,202301010000_79,1,0 days 01:38:28,0 days
3,202301010000_105,202301010000_80,1,0 days 01:58:25,0 days
4,202301010000_106,202301010000_81,1,0 days 06:55:03,0 days
...,...,...,...,...,...
6369366,202312312358_5,202312312354_5,1,0 days 00:04:34,0 days
6369367,202312312358_6,202312312354_5,1,0 days 00:03:57,0 days
6369368,202312312358_7,202312312354_5,1,0 days 00:00:00,0 days
6369369,202312312358_8,202312312354_7,1,0 days 00:05:31,0 days


In [20]:
clusters_statistics.to_parquet("real-time clusters/clusters_statistics.parquet")

## Slogans


In [17]:
mob_slogan = mob[["alarm_id", "mob_slogan"]]
adsl_slogan = adsl[["alarm_id", "std_probable_cause_no"]]## , "summary"]
ptn_slogan = ptn[["alarm_id", "std_probable_cause_no"]]
sdh_slogan = sdh[["alarm_id", "std_probable_cause_no"]]
pdh_slogan = pdh[["alarm_id", "std_probable_cause_no"]]


In [24]:
mob["mob_slogan"]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
           ... 
28176439    NaN
28176440    NaN
28176441    NaN
28176442    NaN
28176443    NaN
Name: mob_slogan, Length: 28176444, dtype: category
Categories (1142, object): ['', 'a', 'a incendio', 'a min batteria', ..., 'wi alllarme porta', 'wi guasto grave condizionamento', 'wi guasto sistema di alimentazione', 'wi mancanza rete']

In [23]:
mob_slogan

,alarm_id,mob_slogan
0,EFM;63b098b01a65d200011620f8,NaN
1,EFM;63b0b476c623b40001a68b45,NaN
2,EFM;63b0b5a8c623b40001a68b9b,NaN
3,EFM;63b0b5c6c623b40001a68ba2,NaN
4,EFM;63b0b675c623b40001a68c03,NaN
...,...,...
28176439,EFM;6591f1f639628d000137853b,NaN
28176440,EFM;6591f1f739628d000137854e,NaN
28176441,EFM;6591f1f739628d0001378554,NaN
28176442,EFM;6591f1f939628d0001378585,NaN


In [11]:
mob_slogan.rename(columns={"mob_slogan": "slogan"}, inplace=True)


C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_15288\753721231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mob_slogan.rename(columns={"mob_slogan": "slogan"}, inplace=True)


In [14]:
mob_slogan

,alarm_id,slogan
0,EFM;63b098b01a65d200011620f8,NaN
1,EFM;63b0b476c623b40001a68b45,NaN
2,EFM;63b0b5a8c623b40001a68b9b,NaN
3,EFM;63b0b5c6c623b40001a68ba2,NaN
4,EFM;63b0b675c623b40001a68c03,NaN
...,...,...
28176439,EFM;6591f1f639628d000137853b,NaN
28176440,EFM;6591f1f739628d000137854e,NaN
28176441,EFM;6591f1f739628d0001378554,NaN
28176442,EFM;6591f1f939628d0001378585,NaN


In [9]:
alarms_slogan = pd.concat([mob_slogan, adsl_slogan, ptn_slogan, sdh_slogan, pdh_slogan])

In [10]:
alarms_slogan

,alarm_id,slogan,std_probable_cause_no
0,EFM;63b098b01a65d200011620f8,NaN,NaN
1,EFM;63b0b476c623b40001a68b45,NaN,NaN
2,EFM;63b0b5a8c623b40001a68b9b,NaN,NaN
3,EFM;63b0b5c6c623b40001a68ba2,NaN,NaN
4,EFM;63b0b675c623b40001a68c03,NaN,NaN
...,...,...,...
408772,EFMTX;65913ce9c23c110001552471,NaN,tunnel aps degraded downgrade of aps tunnel pr...
408773,EFMTX;65913e2cc23c110001552527,NaN,tunnel aps degraded downgrade of aps tunnel pr...
408774,EFMTX;6530d7a8eedb1500019c416e,NaN,opttxlow reducedtransmitteroutputpower
408775,EFMTX;658cd3bf975f510001572fd9,NaN,tunnel aps degraded downgrade of aps tunnel pr...


In [40]:
alarms_slogan.to_parquet("alarms datasets/alarms_slogan.parquet")

In [41]:
import pandas as pd

# Creazione di un DataFrame di esempio
data = {'colonna': [1, 2, 3, 4, 1, 2, 3]}
df = pd.DataFrame(data)

# Ottenere solo gli elementi unici della colonna 'colonna'
elementi_unici = df['colonna'].unique()

# Creare un nuovo DataFrame con gli elementi unici della colonna 'colonna'
df_elementi_unici = pd.DataFrame({'colonna': elementi_unici})

print(df_elementi_unici)


   colonna
0        1
1        2
2        3
3        4
